# Document Creation for NLP Tasks
This notebook loads the Google reviews data and creates two distinct documents for each review:
1. **Review Document:** The raw text of the customer review.
2. **Business Document:** A concatenation of the business name, category, and description.

In [1]:
import pandas as pd

## 1. Load the data

In [3]:
df = pd.read_csv('/Users/yumin/Documents/GitHub/TikTok-TechJam-2025/data_gpt_labeler/final_data_labeled_1.csv')

In [4]:
df.head()

,Unnamed: 0,rating,text,business_name,business_category,business_description,_id,policy_label
0,0,5,My husband took me here for my birthday! The ...,Buona Sera,"['Italian restaurant', 'Restaurant']",Casual trattoria serving familiar Italian entr...,1.089786951023629e+20_1507964592497,1
1,1,4,was a great place. Now closed. Too bad. We'll...,Buona Sera,"['Italian restaurant', 'Restaurant']",Casual trattoria serving familiar Italian entr...,1.124085651879834e+20_1519880201044,1
2,2,5,"cozy, great food. love how you can sign your b...",Buona Sera,"['Italian restaurant', 'Restaurant']",Casual trattoria serving familiar Italian entr...,1.1571615809204435e+20_1490033397528,1
3,3,4,"Not your average little Italian joint, don't e...",Buona Sera,"['Italian restaurant', 'Restaurant']",Casual trattoria serving familiar Italian entr...,1.181306411818766e+20_1511165172600,1
4,4,3,Service was great and the mozzarella fritta an...,Buona Sera,"['Italian restaurant', 'Restaurant']",Casual trattoria serving familiar Italian entr...,1.1430211684992369e+20_1505179108178,1


## 2. Create Review and Business Documents

In [5]:
df['review_document'] = df['text'].astype(str)
df['business_document'] = df['business_name'].fillna('') + ' ' + df['business_category'].fillna('') + ' ' + df['business_description'].fillna('')
df[['review_document', 'business_document']].head()

,review_document,business_document
0,My husband took me here for my birthday! The ...,"Buona Sera ['Italian restaurant', 'Restaurant'..."
1,was a great place. Now closed. Too bad. We'll...,"Buona Sera ['Italian restaurant', 'Restaurant'..."
2,"cozy, great food. love how you can sign your b...","Buona Sera ['Italian restaurant', 'Restaurant'..."
3,"Not your average little Italian joint, don't e...","Buona Sera ['Italian restaurant', 'Restaurant'..."
4,Service was great and the mozzarella fritta an...,"Buona Sera ['Italian restaurant', 'Restaurant'..."


## 3. Display a single example

In [6]:
print('Review Document:')
print(df['review_document'].iloc[0])
print('---')
print('Business Document:')
print(df['business_document'].iloc[0])

Review Document:
My husband took me here for my birthday!  The best meal we have had here! The food is fresh and delicious!  They have a large selection of wine. It's is a very peaceful,  romantic setting.  I loved it. We will definitely come back!
---
Business Document:
Buona Sera ['Italian restaurant', 'Restaurant'] Casual trattoria serving familiar Italian entrees in a family-friendly setting with funky decor.


## 4. Create the Corpus

In [7]:
corpus = []
for index, row in df.iterrows():
    corpus.append(row['review_document'])     
    corpus.append(row['business_document']) 
print(f'Total number of documents in the corpus: {len(corpus)}')
print('---')
print('First 4 documents in the corpus:')
for doc in corpus[:4]:
    print(doc)    
print('---')

Total number of documents in the corpus: 20000
---
First 4 documents in the corpus:
My husband took me here for my birthday!  The best meal we have had here! The food is fresh and delicious!  They have a large selection of wine. It's is a very peaceful,  romantic setting.  I loved it. We will definitely come back!
Buona Sera ['Italian restaurant', 'Restaurant'] Casual trattoria serving familiar Italian entrees in a family-friendly setting with funky decor.
was a great place.  Now closed. Too bad. We'll miss having comfort food from Buona Seras
Buona Sera ['Italian restaurant', 'Restaurant'] Casual trattoria serving familiar Italian entrees in a family-friendly setting with funky decor.
---


## 5. Preprocess Corpus for LDA

In [8]:
import nltk 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):    
    tokens = word_tokenize(text.lower())    
    return [lemmatizer.lemmatize(w) for w in tokens if w.isalpha() and w not in stop_words]
processed_corpus = [preprocess_text(doc) for doc in corpus]

[nltk_data] Downloading package punkt to /Users/yumin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/yumin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/yumin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 6. Create Dictionary and Corpus for LDA

In [9]:
from gensim import corpora
dictionary = corpora.Dictionary(processed_corpus)
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

## 7. Train the LDA Model

In [10]:
from gensim.models import LdaModel
lda_model = LdaModel(bow_corpus, num_topics=100, id2word=dictionary, passes=15)
for idx, topic in lda_model.print_topics(-1):    print(f'Topic: {idx} Words: {topic}')

Topic: 0 Words: 0.498*"range" + 0.220*"snug" + 0.141*"neighborhood" + 0.012*"non" + 0.011*"trailhead" + 0.007*"confusing" + 0.004*"choosing" + 0.004*"shortest" + 0.001*"gated" + 0.000*"musbi"
Topic: 1 Words: 0.094*"starbucks" + 0.078*"bite" + 0.078*"coffeehouse" + 0.077*"light" + 0.076*"signature" + 0.076*"roast" + 0.075*"wifi" + 0.075*"chain" + 0.075*"known" + 0.074*"availability"
Topic: 2 Words: 0.177*"please" + 0.102*"city" + 0.081*"broad" + 0.072*"cashier" + 0.050*"reward" + 0.047*"purchase" + 0.041*"ranch" + 0.040*"onolicious" + 0.023*"unlike" + 0.012*"answering"
Topic: 3 Words: 0.235*"taste" + 0.231*"like" + 0.133*"ice" + 0.120*"much" + 0.079*"cream" + 0.058*"server" + 0.033*"flavor" + 0.018*"butter" + 0.016*"treat" + 0.009*"question"
Topic: 4 Words: 0.247*"kitchen" + 0.117*"super" + 0.113*"experience" + 0.083*"wonderful" + 0.043*"quickly" + 0.040*"various" + 0.036*"located" + 0.027*"ate" + 0.021*"sign" + 0.021*"incredible"
Topic: 5 Words: 0.274*"topping" + 0.242*"valley" + 0.055

## 8. Visualize the Topics

In [12]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, bow_corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
1     -0.098941  0.054031       1        1  4.831012
67    -0.332780 -0.040728       2        1  4.061421
54     0.006328  0.303844       3        1  3.069184
73    -0.324027 -0.028115       4        1  2.584119
72    -0.362095 -0.059623       5        1  2.431899
...         ...       ...     ...      ...       ...
37     0.109583 -0.149673      96        1  0.197632
2      0.121355 -0.168402      97        1  0.175739
63     0.120334 -0.166685      98        1  0.166267
79     0.134943 -0.188458      99        1  0.145211
97     0.136768 -0.191011     100        1  0.128364

[100 rows x 5 columns], topic_info=           Term          Freq         Total  Category  logprob  loglift
25   restaurant  17326.000000  17326.000000   Default  30.0000  30.0000
56         good   3123.000000   3123.000000   Default  29.0000  29.0000
865    offering   3546.000000   3546.000000   Default  28.0000  28.0000
192      noodle   2788.000000   2788.000000   Default  27.0000  27.0000
138        shop   2518.000000   2518.000000   Default  26.0000  26.0000
..          ...           ...           ...       ...      ...      ...
15      setting      0.017087   1049.455135  Topic100  -9.8785  -4.3674
16         took      0.017087    194.364427  Topic100  -9.8785  -2.6811
17         wine      0.017087     47.779085  Topic100  -9.8785  -1.2780
19       casual      0.017087   2557.116730  Topic100  -9.8785  -5.2580
20        decor      0.017087     74.347382  Topic100  -9.8785  -1.7201

[4218 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1463     28  0.984859        able
496      33  0.306193  absolutely
496      38  0.684431  absolutely
324      82  0.908514          ac
1421     44  0.637204  acceptable
...     ...       ...         ...
2396     76  0.814098        york
1896     46  0.946172       young
1810     33  0.970734         yum
1420     24  0.987431       yummy
466      89  0.807542        zero

[2050 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 68, 55, 74, 73, 63, 99, 78, 96, 94, 81, 51, 56, 23, 14, 92, 54, 22, 57, 61, 88, 62, 59, 69, 97, 32, 25, 47, 95, 85, 87, 43, 44, 7, 100, 93, 35, 45, 11, 76, 29, 5, 65, 4, 66, 75, 34, 39, 70, 60, 72, 90, 17, 15, 48, 84, 40, 27, 91, 86, 52, 53, 16, 13, 37, 50, 41, 19, 83, 89, 10, 71, 9, 58, 24, 82, 21, 18, 28, 20, 31, 79, 12, 49, 36, 77, 30, 33, 42, 26, 1, 67, 6, 8, 46, 38, 3, 64, 80, 98])

## 9. Compute Similarity Score

In [13]:
import numpy as np 
from scipy.spatial.distance import cosine
def get_lda_vector(text):    
    processed_text = preprocess_text(text)    
    bow_vector = dictionary.doc2bow(processed_text)    
    lda_vector = lda_model.get_document_topics(bow_vector, minimum_probability=0.0)    
    dense_vector = np.zeros(lda_model.num_topics)    
    for topic_num, prop_topic in lda_vector:        
        dense_vector[topic_num] = prop_topic    
    return dense_vector
    
def calculate_cosine_similarity(vec1, vec2):    
    return 1 - cosine(vec1, vec2)
    
# Example with the first review
sample_review = df['review_document'].iloc[0]
sample_business = df['business_document'].iloc[0]
review_vector = get_lda_vector(sample_review)
business_vector = get_lda_vector(sample_business)
similarity_score = calculate_cosine_similarity(review_vector, business_vector)
print(f'Review: {sample_review}')
print(f'Business: {sample_business}')
print(f'Similarity Score (Trustworthiness): {similarity_score:.4f}')

Review: My husband took me here for my birthday!  The best meal we have had here! The food is fresh and delicious!  They have a large selection of wine. It's is a very peaceful,  romantic setting.  I loved it. We will definitely come back!
Business: Buona Sera ['Italian restaurant', 'Restaurant'] Casual trattoria serving familiar Italian entrees in a family-friendly setting with funky decor.
Similarity Score (Trustworthiness): 0.1355
